<a href="https://colab.research.google.com/github/SampMark/Computational-Thinking/blob/main/Problem_Set1_space_cows.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Problem Set1**: Transportando Vacas pelo Espaço (Space Cows Introduction)

Os Aucks, uma espécie de alienígenas bioengenheiros superinteligentes, chegaram à Terra e criaram novas espécies de animais de fazenda. Após realizarem seus experimentos, eles precisam transportar esses animais de volta ao planeta natal, Aurock. O exercício proposto visa implementar algoritmos para se determinar a melhor forma de transportar vacas mutantes pelo espaço, respeitando as restrições de peso da espaçonave e minimizando o número de viagens. O objetivo é transportar todas as vacas utilizando o menor número de viagens possível.

**Contexto do problema**:
* As vacas criadas pelos alienígenas têm pesos inteiros em toneladas.
* Nome único de cada vaca.
* A espaçonave tem um limite de peso máximo por viagem.

**Tarefas envolvidas**:

1) **Parte A**: Implementar o algoritmo guloso (Greedy), capaz de  escolher a vaca mais pesada que ainda cabe na nave até atingir o limite, prosseguindo com as demais vacas em uma nova viagem.

2) **Parte B**: Implementar o algoritmo de força bruta (Brute Force), que deverá ser capaz de testar todas as combinações possíveis de particionamento das vacas de modo a encontrar a solução ótima, com o menor número de viagens.

3) **Parte C**: Comparar os algoritmos, medindo o desempenho (tempo de execução) e a eficácia (número de viagens) entre ambos.

In [37]:
!pip install pandas

In [38]:
import pandas as pd
import requests
import time
# Fazer o upload do arquivo ps1_partition.py (fornecido)
# para a pasta raiz (/content/) no Google Colab
from ps1_partition import get_partitions

## **Aplicações práticas em problemas reais**

O código desenvolvido a seguir pode ser adaptado para resolver problemas reais em logística e transporte de cargas, tais como:

* **Gerenciamento de frota e entregas**: otimizar o uso de caminhões para transporte de cargas em empresas de logística, respeitando limites de peso e volumes. O algoritmo guloso pode ser usado em cenários em que a velocidade de cálculo seja mais importante do que a solução ótima, como em operações diárias.

* **Planejamento de viagens aéreas**: para distribuir cargas em aviões com limite de peso e volume, considerando a segurança operacional.

* **Armazenamento de estoques em galpões**: para alocar itens em paletes ou contêineres, garantindo que os limites de peso ou tamanho sejam respeitados.

* **Distribuição de Recursos em Situações de Emergência**: para planejar o transporte de alimentos, água e suprimentos para áreas afetadas por desastres naturais, minimizando viagens e otimizando o uso de veículos.

* **Planejamento de eventos**: no transporte de equipamentos pesados para locais de eventos, considerando restrições de transporte e logística.

In [39]:
# Os dados sobre as vacas estão armazenados no GitHub, 'ps1_cow_data.txt'
# contendo os pares no formato separado por vírgulas 'vaga,peso por linha'.
cow_dict = pd.read_table('https://raw.githubusercontent.com/SampMark/files/refs/heads/main/ps1_cow_data.txt')
cow_dict.head(10)

,"Cow,Weight"
0,"Maggie,3"
1,"Herman,7"
2,"Betsy,9"
3,"Oreo,6"
4,"Moo Moo,3"
5,"Milkshake,2"
6,"Millie,5"
7,"Lola,2"
8,"Florence,2"
9,"Henrietta,9"


## **Cria um dicionário com nomes das vacas como chaves e pesos como valores**

In [40]:
# URL do arquivo de dados
data_url = 'https://raw.githubusercontent.com/SampMark/files/refs/heads/main/ps1_cow_data.txt'

def load_cows_from_url(url):
    """
    Faz o download do arquivo da URL e carrega os dados como dicionário.

    Args:
      url: URL para o arquivo de dados.

    Returns:
      Dicionário com nomes das vacas como chaves e pesos como valores.
    """
    cow_dict = {}
    response = requests.get(url)
    response.raise_for_status()  # Garante que o download foi bem-sucedido

    # Pula a primeira linha de cabeçalho
    lines = response.text.strip().split('\n')
    for line in lines[1:]: # Começa a iterar a partir da segunda linha
    #for line in response.text.strip().split('\n'):
        name, weight = line.split(',')
        cow_dict[name.strip()] = int(weight.strip())
    return cow_dict

# Substituir o método de carregamento de arquivo
load_cows = load_cows_from_url

## **Parte 1: Greedy Cow Transport**

O algoritmo ganancioso propostoso a seguir escolherá sempre a vaca mais pesada que ainda cabe no espaço disponível da aeronave. O objetivo da primeira parte do exercíco proposto usando a função `greedy_cow_transport` é transportar "vacas pelo espaço" utilizando o menor número possível de viagens, seguindo uma abordagem gananciosa (*greedy*).

Após preencher o espaço ou não, é possível adicionar mais vacas e uma nova viagem é iniciada com as vacas restantes. A limitação da abordagem gananciosa é que ela não garante uma solução ótima em todos os casos, pois não considera todas as combinações possíveis.

**Exemplo**:
Para o limite de peso da aeronave de 10 toneladas e vacas com pesos {"Jesse": 6, "Maybel": 3, "Callie": 2, "Maggie": 5}, o algoritmo primeiro escolhe "Jesse" (6 toneladas), depois "Maybel" (3 toneladas) para a mesma viagem. Na próxima viagem, leva "Maggie" (5 toneladas) e "Callie" (2 toneladas). O resultado seria [['Jesse', 'Maybel'], ['Maggie', 'Callie']].



In [41]:
def greedy_cow_transport(cows, limit=10):
    """
    Transporta vacas em uma nave espacial com limite de peso usando um algoritmo guloso.

    Args:
      cows: Um dicionário mapeando nomes de vacas para seus pesos.
      limit: O limite de peso da nave espacial.

    Returns:
      Uma lista de listas, onde cada lista interna representa uma viagem e contém os
      nomes das vacas levadas nessa viagem.
    """
    trips = []  # Inicializa uma lista para armazenar as viagens
    remaining_cows = cows.copy()  # Cria uma cópia do dicionário cow_dict para evitar modificá-lo
    sorted_cows = sorted(remaining_cows.items(), key=lambda x: x[1], reverse=True)  # Ordena as vacas por peso em ordem decrescente

    # Continua enquanto houver vacas restantes para transportar
    while remaining_cows:
        current_trip = []  # Inicializa uma lista para a viagem atual
        current_weight = 0  # Inicializa o peso atual da viagem
        to_remove = [] # Inicializa uma lista para armazenar as vacas a serem removidas no final da iteração

        # Itera sobre as vacas ordenadas por peso
        for cow, weight in sorted_cows:
            # Se a vaca ainda não foi transportada e couber na viagem atual
            if cow in remaining_cows and weight + current_weight <= limit:
                current_trip.append(cow)  # Adiciona a vaca à viagem atual
                current_weight += weight  # Atualiza o peso atual da viagem
                to_remove.append(cow) # Adiciona a vaca na lista para remoção posterior

        # Remove as vacas que foram adicionadas à viagem atual
        for cow in to_remove:
            remaining_cows.pop(cow)

        trips.append(current_trip)  # Adiciona a viagem atual à lista de viagens

    return trips  # Retorna a lista de viagens

In [42]:
def is_valid_partition(partition, cows, limit):
    """
    Valida se uma partição de viagens respeita o limite de peso da nave.

    Args:
      partition: Uma partição representada como uma lista de listas (viagens).
      cows: Um dicionário mapeando os nomes das vacas para seus pesos.
      limit: O limite de peso da nave.

    Returns:
      True se todas as viagens na partição forem válidas, False caso contrário.
    """
    for trip in partition:
        total_weight = sum(cows[cow] for cow in trip)
        if total_weight > limit:
            return False
    return True

## **Parte 2: Brute Force Cow Transport**

O algoritmo considera todas as combinações possíveis de particionamento das vacas em viagens e escolhe aquela que atende às restrições (limite de peso) com o menor número de viagens. O objetivo é encontrar a **solução ótima** (menor número de viagens) usando o algoritmo de "força bruta", definido em `brute_force_cow_transport`.

Apesar de garantir a solução ótima, o algoritmo de força bruta é mais lento computacinalmente, especialmente com grandes conjuntos de dados, devido ao número exponencial de combinações.

**Exemplo**:
Para as mesmas vacas e limite de peso, o algoritmo tenta todas as combinações de particionamento, como [['Jesse', 'Callie'], ['Maybel', 'Maggie']], até encontrar a melhor.


In [43]:
def brute_force_cow_transport(cows, limit=10):
    """
    Encontra a solução ótima para transportar vacas em uma nave espacial com um
    limite de peso usando força bruta.

    Args:
      cows: Um dicionário mapeando nomes de vacas para seus pesos.
      limit: O limite de peso da nave espacial.

    Returns:
      Uma lista de listas, onde cada lista interna representa uma viagem e contém os
      nomes das vacas levadas nessa viagem.
    """
    best_partition = None  # Inicializa a melhor partição como None
    min_trips = float('inf')  # Inicializa o número mínimo de viagens como infinito
    for partition in get_partitions(cows):
        valid = True  # Inicializa a validade da partição como True
        for trip in partition:
            total_weight = sum(cows[cow] for cow in trip)  # Calcula o peso total da viagem
            if total_weight > limit:  # Se o peso total excede o limite
                valid = False  # A partição é inválida
                break  # Sai do loop interno
        if valid and len(partition) < min_trips:  # Se a partição é válida e tem menos viagens que o mínimo atual
            best_partition = partition  # Atualiza a melhor partição
            min_trips = len(partition)  # Atualiza o número mínimo de viagens
    return best_partition  # Retorna a melhor partição

## **Parte 3: Comparação dos Algoritmos: `greedy_cow_transport` *versus* `brute_force_cow_transport`**
A função `compare_cow_transport_algorithms`, como o nome diz, tem por objetivo comparar os algoritmos "ganancioso" e "força bruta" em termos de desempenho (tempo de execução) e número de viagens geradas. Essa etapa avalia os dois métodos com os mesmos dados e limites de peso, medindo o tempo de execução e o número de viagens geradas.
Em suma, o algoritmo ganancioso é mais rápido, mas o de "força bruta" encontra a solução ótima.

In [44]:
def compare_cow_transport_algorithms():
    """
    Compara os algoritmos de transporte de vacas (guloso e força bruta) em termos de:
    - Número mínimo de viagens.
    - Tempo de execução.

    Carrega os dados das vacas, executa ambos os algoritmos e imprime os resultados.
    """
    # Carrega os dados das vacas
    cows = load_cows(data_url)  # Carrega os dados diretamente da URL
    limit = 10  # Limite de peso padrão

    print("Comparando algoritmos de transporte de vacas...")

    # Testa o algoritmo guloso
    start_greedy = time.time()
    greedy_result = greedy_cow_transport(cows, limit)
    end_greedy = time.time()
    greedy_time = end_greedy - start_greedy

    print("\nAlgoritmo Guloso:")
    print(f"Número de viagens: {len(greedy_result)}")
    print(f"Tempo de execução: {greedy_time:.4f} segundos")

    # Testa o algoritmo de força bruta
    start_brute = time.time()
    brute_force_result = brute_force_cow_transport(cows, limit)
    end_brute = time.time()
    brute_force_time = end_brute - start_brute

    print("\nAlgoritmo de Força Bruta:")
    print(f"Número de viagens: {len(brute_force_result)}")
    print(f"Tempo de execução: {brute_force_time:.4f} segundos")

# As funções load_cows, greedy_cow_transport e brute_force_cow_transport
# foram definidas antes de rodar esta função.
compare_cow_transport_algorithms()

Comparando algoritmos de transporte de vacas...

Algoritmo Guloso:
Número de viagens: 6
Tempo de execução: 0.0000 segundos

Algoritmo de Força Bruta:
Número de viagens: 5
Tempo de execução: 0.8813 segundos


## **Ideias para projetos futuros**

Adaptação do algoritmo guloso para adicionar critérios adicionais, como preferência por objetos com maior valor econômico, considerando o volume dos objetos e as restrições de volume e fomato dos compartimentos de cargas.

Criação de gráficos para visualização interativa, mostrando o preenchimento do meio de transponrte em cada viagem, de modo a ajudar os usuários a entenderem as decisões do algoritmo.

Usar modelos preditivos, com integração em machine learning, para estimar demandas futuras e ajustar os parâmetros do algoritmo.
